In [ ]:
import sys
sys.path.append('..')

from database import SessionLocal, engine
from models import Base, User, Channel, CompanyInfo, CompanyReport
from datetime import datetime
import json

# Create all tables
Base.metadata.create_all(bind=engine)

# Create a database session
db = SessionLocal()

In [ ]:
# Create a sample channel
channel = Channel(
    channel_number="CH001",
    channel_name="Sample Channel",
    channel_location="Beijing",
    industry="Finance",
    contact_person="John Doe",
    contact_number="1234567890",
    email="contact@samplechannel.com",
    website="www.samplechannel.com",
    balance=1000.0
)

db.add(channel)
db.commit()
db.refresh(channel)

In [ ]:
# Create level 1 and level 2 users
level1_user = User(
    email="level1@example.com",
    username="level1_user",
    password="hashed_password",  # In real app, this should be properly hashed
    firstname="Level1",
    lastname="Admin",
    is_admin=False,
    role="level_1",
    channel_id=channel.id
)

level2_user1 = User(
    email="level2_1@example.com",
    username="level2_user1",
    password="hashed_password",
    firstname="Level2",
    lastname="User1",
    is_admin=False,
    role="level_2",
    channel_id=channel.id,
    first_level_channel_id=1  # Will be set to level1_user's id
)

level2_user2 = User(
    email="level2_2@example.com",
    username="level2_user2",
    password="hashed_password",
    firstname="Level2",
    lastname="User2",
    is_admin=False,
    role="level_2",
    channel_id=channel.id,
    first_level_channel_id=1  # Will be set to level1_user's id
)

db.add(level1_user)
db.commit()
db.refresh(level1_user)

# Update first_level_channel_id with actual level1_user id
level2_user1.first_level_channel_id = level1_user.id
level2_user2.first_level_channel_id = level1_user.id

db.add(level2_user1)
db.add(level2_user2)
db.commit()
db.refresh(level2_user1)
db.refresh(level2_user2)

In [ ]:
# Create sample company info
companies = [
    CompanyInfo(
        company_name="Company A",
        tax_number="TAX001",
        index_standard_type="Standard A",
        industry="Technology",
        registration_type="LLC",
        taxpayer_nature="Regular",
        upload_year=2023,
        channel_id=channel.id,
        status=True
    ),
    CompanyInfo(
        company_name="Company B",
        tax_number="TAX002",
        index_standard_type="Standard B",
        industry="Retail",
        registration_type="Corporation",
        taxpayer_nature="Regular",
        upload_year=2023,
        channel_id=channel.id,
        status=True
    )
]

for company in companies:
    db.add(company)
db.commit()

In [ ]:
# Create sample reports connected to level2 users
reports = [
    # Reports for level2_user1
    CompanyReport(
        processed_by_user_id=level2_user1.id,
        company_tax_number="TAX001",
        report_type="annual",
        year=2023,
        report_data=json.dumps({"data": "Sample annual report data"})
    ),
    CompanyReport(
        processed_by_user_id=level2_user1.id,
        company_tax_number="TAX001",
        report_type="quarterly",
        year=2023,
        quarter=2,
        report_data=json.dumps({"data": "Sample Q2 report data"})
    ),
    # Reports for level2_user2
    CompanyReport(
        processed_by_user_id=level2_user2.id,
        company_tax_number="TAX002",
        report_type="annual",
        year=2023,
        report_data=json.dumps({"data": "Sample annual report data"})
    ),
    CompanyReport(
        processed_by_user_id=level2_user2.id,
        company_tax_number="TAX002",
        report_type="monthly",
        year=2023,
        month=6,
        report_data=json.dumps({"data": "Sample June report data"})
    )
]

for report in reports:
    db.add(report)
db.commit()

In [ ]:
# Close the database session
db.close()